# Let's download wikipedia in english and basque and prepare for lm training

In [17]:
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm
basque = load_dataset("wikimedia/wikipedia", "20231101.eu", split="train")
basque = basque.shuffle(seed=42)

In [18]:
basque

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 416347
})

In [19]:
english = load_dataset("wikimedia/wikipedia", "20231101.en", split="train", streaming=True)
english = english.shuffle(seed=42, buffer_size=1000000)
iterable_english = iter(english)
num_basque_items = len(basque)
english_list = []
for i in tqdm(range(num_basque_items)):
    next_english_entry = next(iterable_english)
    english_list.append(next_english_entry)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/416347 [00:00<?, ?it/s]

In [20]:
# english_list[0]['text'][0:1000]

In [21]:
eng_dataset = Dataset.from_list(english_list)

In [22]:
eng_dataset

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 416347
})

In [23]:
num_total = num_basque_items
eng_dataset = eng_dataset.select(range(0,num_total))
bas_dataset = basque.select(range(0,num_total))
print(num_total)

416347


In [9]:
from transformers import AutoTokenizer
model_id = "HuggingFaceTB/SmolLM2-135M"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [10]:
eng_texts = [elt['text'] for elt in eng_dataset]
eng_tokenized = tokenizer(eng_texts, max_length=4096, padding=False, truncation=True, return_length=True)
eng_lengths = [len(elt) for elt in eng_tokenized['input_ids']]
sum(eng_lengths)

85130370

In [11]:
bas_texts = [elt['text'] for elt in bas_dataset]
bas_tokenized = tokenizer(bas_texts, max_length=4096, padding=False, truncation=True, return_length=True)
bas_lengths = [len(elt) for elt in bas_tokenized['input_ids']]
sum(bas_lengths)

47907436

In [24]:
eng_dataset_dict = eng_dataset.train_test_split(test_size=0.1, seed=42)
eng_dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [25]:
eng_dataset_test = eng_dataset_dict['test']
# eng_dataset_test.push_to_hub("Neelectric/wikipedia_20231101.en_100k_test")
# eng_dataset_dict.push_to_hub("Neelectric/wikipedia_20231101.en_100k")

In [26]:
bas_dataset_dict = bas_dataset.train_test_split(test_size=0.1, seed=42)
bas_dataset_test = bas_dataset_dict['test']
# bas_dataset_dict.push_to_hub("Neelectric/wikipedia_20231101.eu_100k_test")
# bas_dataset_dict.push_to_hub("Neelectric/wikipedia_20231101.eu_100k")

In [27]:
mixed = eng_dataset_dict
mixed['test'] = bas_dataset_dict['test']
mixed

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 374712
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 41635
    })
})

In [28]:
mixed.push_to_hub(("Neelectric/wikipedia_20231101.374k_en_41k_eu"))

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/datasets/Neelectric/wikipedia_20231101.374k_en_41k_eu/commit/4c3c4c89ef072a16329e6926b4484676c5659064', commit_message='Upload dataset', commit_description='', oid='4c3c4c89ef072a16329e6926b4484676c5659064', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Neelectric/wikipedia_20231101.374k_en_41k_eu', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Neelectric/wikipedia_20231101.374k_en_41k_eu'), pr_revision=None, pr_num=None)